In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium, branca
import requests, json
import branca.colormap as cm

from datetime import datetime, timedelta
from folium import FeatureGroup
from folium.features import GeoJson, GeoJsonTooltip

In [2]:
datum = '12-03-2020' #datetime.now().strftime('%d-%m-%Y')

In [3]:
def createGemeenteGDF():
    dfGemeente = pd.read_csv("../rivm_covid_19_data/rivm_covid_19_time_series/time_series_19-covid-Confirmed_city.csv",
                             usecols=['gemeente_id', 'Gemeentenaam', datum])
    
    geojson = '../geo_data/gemeentes.geojson'
    gdf = gpd.read_file(geojson)
    gdf['code'] = gdf['code'].astype(int)
    gdfMerged  = gdf.merge(dfGemeente, how='left', left_on='code', right_on='gemeente_id')
    
    return(gdfMerged)

In [4]:
def createProvincieGDF():
    dfProvincie = pd.read_csv("../rivm_covid_19_data/rivm_covid_19_time_series/time_series_19-covid-Confirmed_province.csv",
                             usecols=['Provinciecode', 'Provincienaam', datum])
    
    geojson = '../geo_data/provincies.geojson'
    gdf = gpd.read_file(geojson)
    gdf['code'] = gdf['code'].astype(int)
    gdfMerged  = gdf.merge(dfProvincie, how='left', left_on='code', right_on='Provinciecode')
    
    return(gdfMerged)

In [5]:
def createMap():
    m = folium.Map(
        location=[52, 5],
        zoom_start=7,
        tiles='cartodbpositron',
        min_zoom=7,
        max_zoom=10,)
    
    return m

In [6]:
def createColormap():
    df    = createGemeenteGDF()
    colormap = cm.linear.YlOrRd_04.scale(
            df[datum].min(),
            df[datum].max(),)
    
    colormap.caption = ('Besmettingen per provincie op: ' + datum)
    
    return colormap

In [7]:
def createColormapProv():
    df    = createProvincieGDF()
    colormap = cm.linear.YlOrRd_04.scale(
            df[datum].min(),
            df[datum].max(),)
    
    colormap.caption = ('Besmettingen per provincie op: ' + datum)
    
    return colormap

In [8]:
def createTooltip():
    tooltip = GeoJsonTooltip(
                    fields=["gemeentenaam", datum],
                    aliases=["Gemeente:", "Besmettingen:"],
                    localize=True,
                    sticky=False,
                    labels=True,
                    style="""
                        background-color: #F0EFEF;
                        border: 2px solid black;
                        border-radius: 1px;
                        box-shadow: 3px;
                    """,
                )
    return tooltip

In [9]:
def createTooltipProv():
    tooltip = GeoJsonTooltip(
                    fields=["Provincienaam", datum],
                    aliases=["Provincie:", "Besmettingen:"],
                    localize=True,
                    sticky=False,
                    labels=True,
                    style="""
                        background-color: #F0EFEF;
                        border: 2px solid black;
                        border-radius: 1px;
                        box-shadow: 3px;
                    """,
                )
    return tooltip

In [10]:
def addGeoJson():
    colormap = createColormap()
    tooltip  = createTooltip()
    gdf       = createGemeenteGDF()
    
    g = folium.GeoJson(
                gdf,
                name="Gemeentes",
                style_function=lambda x: {
                    "fillColor": colormap(x["properties"][datum])
                    if x["properties"][datum] > 0
                    else "#E7E4E3",
                    "color": "black",
                    "weight": 1,
                    "fillOpacity": 0.75,
                },
                tooltip=tooltip
            )
    
    return g

In [11]:
def addGeoJsonProv():
    colormap = createColormapProv()
    tooltip  = createTooltipProv()
    gdf       = createProvincieGDF()
    
    g = folium.GeoJson(
                gdf,
                name="Provincies",
                style_function=lambda x: {
                    "fillColor": colormap(x["properties"][datum])
                    if x["properties"][datum] is not None
                    else "#E7E4E3",
                    "color": "black",
                    "weight": 1,
                    "fillOpacity": 0.75,
                },
                tooltip=tooltip,
                show=False
            )
    
    return g

In [12]:
def createLegend():
    legend = folium.LayerControl(collapsed=False)
    
    return legend

In [13]:
def makeMap():
    m = createMap()
    colormap     = createColormap()
    geoJson      = addGeoJson()
    geoJson2     = addGeoJsonProv()
    legend       = createLegend()
    
    #feature_group = FeatureGroup(name='Gemeentes')
    
    geoJson.add_to(m)
    geoJson2.add_to(m)
    m.add_child(colormap)
    m.add_child(legend)

    return m

In [14]:
m = makeMap()

In [15]:
m

In [16]:
#m.save("kaart_" + datum + ".html") 